In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from numpy.fft import fft

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"] = (30, 20)

## process_file() takes a string with a path to the .csv


def process_file(pathname):
    res = pd.read_csv(pathname, sep=',', header=0, index_col=None, names=[
                      'time', 'seconds', 'z', 'y', 'x'])
    return res


def butterworth(df):
    nyquist_frequency = 0.5 * 50

    low = 0.5 / nyquist_frequency
    high = 2.2 / nyquist_frequency

    b, a = signal.butter(3, Wn=[low, high], btype='bandpass')

    df['x'] = signal.filtfilt(b, a, df['x'])
    df['y'] = signal.filtfilt(b, a, df['y'])
    df['z'] = signal.filtfilt(b, a, df['z'])

    return df


In [2]:
slp_5mins_bw = process_file('processed_data/srp_5min_bw.csv')

dlp_upstairs_1_bw = process_file('processed_data/dlp_upstairs_1_bw.csv')
dlp_upstairs_2_bw = process_file('processed_data/dlp_upstairs_2_bw.csv')
dlp_downstairs_1_bw = process_file('processed_data/dlp_downstairs_1_bw.csv')
dlp_downstairs_2_bw = process_file('processed_data/dlp_downstairs_2_bw.csv')

mrp_upstairs_1_bw = butterworth(process_file(
    'processed_data/mrp_upstairs_16steps.csv'))
m_up24_1_bw = process_file('processed_data/m_up24_1_bw.csv')
m_up24_2_bw = process_file('processed_data/m_up24_2_bw.csv')
m_up24_3_bw = process_file('processed_data/m_up24_3_bw.csv')
m_up24_4_bw = process_file('processed_data/m_up24_4_bw.csv')
m_up24_5_bw = process_file('processed_data/m_up24_5_bw.csv')

mrp_downstairs_1 = butterworth(process_file(
    'processed_data/mrp_downstairs_16steps.csv'))
m_down24_1_bw = process_file('processed_data/m_down24_1_bw.csv')
m_down24_2_bw = process_file('processed_data/m_down24_2_bw.csv')
m_down24_3_bw = process_file('processed_data/m_down24_3_bw.csv')
m_down24_4_bw = process_file('processed_data/m_down24_4_bw.csv')
m_down24_5_bw = process_file('processed_data/m_down24_5_bw.csv')


s_up_1_bw = process_file('processed_data/srp_up_1_bw.csv')
s_up_2_bw = process_file('processed_data/srp_up_2_bw.csv')
s_up_3_bw = process_file('processed_data/srp_up_3_bw.csv')
s_up_4_bw = process_file('processed_data/srp_up_4_bw.csv')
s_up_5_bw = process_file('processed_data/srp_up_5_bw.csv')
s_up_6_bw = process_file('processed_data/srp_up_6_bw.csv')
s_up_7_bw = process_file('processed_data/srp_up_7_bw.csv')
s_up_8_bw = process_file('processed_data/srp_up_8_bw.csv')
s_up_9_bw = process_file('processed_data/srp_up_9_bw.csv')
s_up_10_bw = process_file('processed_data/srp_up_10_bw.csv')

s_down_1_bw = process_file('processed_data/srp_down_1_bw.csv')
s_down_2_bw = process_file('processed_data/srp_down_2_bw.csv')
s_down_3_bw = process_file('processed_data/srp_down_3_bw.csv')
s_down_4_bw = process_file('processed_data/srp_down_4_bw.csv')
s_down_5_bw = process_file('processed_data/srp_down_5_bw.csv')
s_down_6_bw = process_file('processed_data/srp_down_6_bw.csv')
s_down_7_bw = process_file('processed_data/srp_down_7_bw.csv')
s_down_8_bw = process_file('processed_data/srp_down_8_bw.csv')
s_down_9_bw = process_file('processed_data/srp_down_9_bw.csv')
s_down_10_bw = process_file('processed_data/srp_down_10_bw.csv')

dlp_upstairs_1_bw['movement'] = 'upstairs'
dlp_upstairs_2_bw['movement'] = 'upstairs'
dlp_downstairs_1_bw['movement'] = 'downstairs'
dlp_downstairs_2_bw['movement'] = 'downstairs'
mrp_upstairs_1_bw['movement'] = 'upstairs'
m_up24_1_bw['movement'] = 'upstairs'
m_up24_2_bw['movement'] = 'upstairs'
m_up24_3_bw['movement'] = 'upstairs'
m_up24_4_bw['movement'] = 'upstairs'
m_up24_5_bw['movement'] = 'upstairs'
mrp_downstairs_1['movement'] = 'downstairs'
m_down24_1_bw['movement'] = 'downstairs'
m_down24_2_bw['movement'] = 'downstairs'
m_down24_3_bw['movement'] = 'downstairs'
m_down24_4_bw['movement'] = 'downstairs'
m_down24_5_bw['movement'] = 'downstairs'

s_up_1_bw['movement'] = 'upstairs'
s_up_2_bw['movement'] = 'upstairs'
s_up_3_bw['movement'] = 'upstairs'
s_up_4_bw['movement'] = 'upstairs'
s_up_5_bw['movement'] = 'upstairs'
s_up_6_bw['movement'] = 'upstairs'
s_up_7_bw['movement'] = 'upstairs'
s_up_8_bw['movement'] = 'upstairs'
s_up_9_bw['movement'] = 'upstairs'
s_up_10_bw['movement'] = 'upstairs'

s_down_1_bw['movement'] = 'downstairs'
s_down_2_bw['movement'] = 'downstairs'
s_down_3_bw['movement'] = 'downstairs'
s_down_4_bw['movement'] = 'downstairs'
s_down_5_bw['movement'] = 'downstairs'
s_down_6_bw['movement'] = 'downstairs'
s_down_7_bw['movement'] = 'downstairs'
s_down_8_bw['movement'] = 'downstairs'
s_down_9_bw['movement'] = 'downstairs'
s_down_10_bw['movement'] = 'downstairs'

dlp_upstairs_1_split = np.array_split(dlp_upstairs_1_bw, 4)
dlp_upstairs_2_split = np.array_split(dlp_upstairs_2_bw, 4)
dlp_downstairs_1_split = np.array_split(dlp_downstairs_1_bw, 4)
dlp_downstairs_2_split = np.array_split(dlp_downstairs_2_bw, 4)
mrp_upstairs_1_split = np.array_split(mrp_upstairs_1_bw, 4)
m_up24_1_bw_split = np.array_split(m_up24_1_bw, 4)
m_up24_2_bw_split = np.array_split(m_up24_2_bw, 4)
m_up24_3_bw_split = np.array_split(m_up24_3_bw, 4)
m_up24_4_bw_split = np.array_split(m_up24_4_bw, 4)
m_up24_5_bw_split = np.array_split(m_up24_5_bw, 4)
mrp_downstairs_1_split = np.array_split(mrp_downstairs_1, 4)
m_down24_1_bw_split = np.array_split(m_down24_1_bw, 4)
m_down24_2_bw_split = np.array_split(m_down24_2_bw, 4)
m_down24_3_bw_split = np.array_split(m_down24_3_bw, 4)
m_down24_4_bw_split = np.array_split(m_down24_4_bw, 4)
m_down24_5_bw_split = np.array_split(m_down24_5_bw, 4)

s_up_1_bw_split = np.array_split(s_up_1_bw, 4)
s_up_2_bw_split = np.array_split(s_up_2_bw, 4)
s_up_3_bw_split = np.array_split(s_up_3_bw, 4)
s_up_4_bw_split = np.array_split(s_up_4_bw, 4)
s_up_5_bw_split = np.array_split(s_up_5_bw, 4)
s_up_6_bw_split = np.array_split(s_up_6_bw, 4)
s_up_7_bw_split = np.array_split(s_up_7_bw, 4)
s_up_8_bw_split = np.array_split(s_up_8_bw, 4)
s_up_9_bw_split = np.array_split(s_up_9_bw, 4)
s_up_10_bw_split = np.array_split(s_up_10_bw, 4)

s_down_1_bw_split = np.array_split(s_down_1_bw, 4)
s_down_2_bw_split = np.array_split(s_down_2_bw, 4)
s_down_3_bw_split = np.array_split(s_down_3_bw, 4)
s_down_4_bw_split = np.array_split(s_down_4_bw, 4)
s_down_5_bw_split = np.array_split(s_down_5_bw, 4)
s_down_6_bw_split = np.array_split(s_down_6_bw, 4)
s_down_7_bw_split = np.array_split(s_down_7_bw, 4)
s_down_8_bw_split = np.array_split(s_down_8_bw, 4)
s_down_9_bw_split = np.array_split(s_down_9_bw, 4)
s_down_10_bw_split = np.array_split(s_down_10_bw, 4)

tmove = []
tdata = []

for i in [dlp_upstairs_1_split, dlp_upstairs_2_split, dlp_downstairs_1_split, dlp_downstairs_2_split, mrp_upstairs_1_split, m_up24_1_bw_split, m_up24_2_bw_split, m_up24_3_bw_split, m_up24_4_bw_split, m_up24_5_bw_split, mrp_downstairs_1_split, m_down24_1_bw_split, m_down24_2_bw_split, m_down24_3_bw_split, m_down24_4_bw_split, m_down24_5_bw_split, s_up_1_bw_split, s_up_2_bw_split, s_up_3_bw_split, s_up_4_bw_split, s_up_5_bw_split, s_up_6_bw_split, s_up_7_bw_split, s_up_8_bw_split, s_up_9_bw_split, s_up_10_bw_split, s_down_1_bw_split, s_down_2_bw_split, s_down_3_bw_split, s_down_4_bw_split, s_down_5_bw_split, s_down_6_bw_split, s_down_7_bw_split, s_down_8_bw_split, s_down_9_bw_split, s_down_10_bw_split]:
    for j in range(len(i) - 1):
        temp = i[j]

        # if ((temp['movement'] == 'walking').all()):
            # tmove.append('walking')
        if ((temp['movement'] == 'upstairs').all()):
            tmove.append('upstairs')
        if ((temp['movement'] == 'downstairs').all()):
            tmove.append('downstairs')

        temp['combined'] = temp['y'] + temp['z']
        tdata.append(np.asarray(temp['combined']))

training_data = pd.DataFrame(tmove)
training_data['joined'] = pd.Series(tdata, index=training_data.index)
training_data

training_data.rename(columns={0: 'movement'}, inplace=True)
training_data = pd.concat([training_data['movement'], training_data.pop(
    'joined').apply(pd.Series)], axis=1).dropna(axis=1)
training_data


,movement,0,1,2,3,4,5,6,7,8,...,83,84,85,86,87,88,89,90,91,92
0,upstairs,0.939890,1.336056,1.687374,1.960024,2.125367,2.161996,2.057994,1.813111,1.440323,...,-1.528790,-1.983224,-2.388069,-2.717393,-2.941399,-3.030885,-2.964149,-2.733299,-2.347548,-1.833174
1,upstairs,-2.918936,-2.707874,-2.313125,-1.782125,-1.175569,-0.555401,0.024901,0.526104,0.925730,...,0.609138,0.027509,-0.531105,-1.027495,-1.426952,-1.705235,-1.852168,-1.872014,-1.780947,-1.602765
2,upstairs,-0.389383,-0.614847,-0.869202,-1.126962,-1.356867,-1.528410,-1.618357,-1.614542,-1.516473,...,3.161443,2.990278,2.693271,2.264739,1.714643,1.068741,0.364760,-0.353732,-1.043302,-1.664654
3,upstairs,0.939890,1.336056,1.687374,1.960024,2.125367,2.161996,2.057994,1.813111,1.440323,...,-1.528790,-1.983224,-2.388069,-2.717393,-2.941399,-3.030885,-2.964149,-2.733299,-2.347548,-1.833174
4,upstairs,-2.918936,-2.707874,-2.313125,-1.782125,-1.175569,-0.555401,0.024901,0.526104,0.925730,...,0.609138,0.027509,-0.531105,-1.027495,-1.426952,-1.705235,-1.852168,-1.872014,-1.780947,-1.602765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,downstairs,0.961067,1.242607,1.564987,1.880644,2.129939,2.246994,2.168339,1.844163,1.249948,...,5.626502,6.480819,6.937526,6.971435,6.588364,5.824931,4.745915,3.438742,2.005219,0.551390
104,downstairs,-0.165009,1.640715,3.456462,5.145364,6.573992,7.621119,8.185920,8.196382,7.618079,...,-3.893523,-3.168845,-2.315959,-1.388517,-0.447529,0.441799,1.217761,1.832783,2.262719,2.510229
105,downstairs,-0.115738,0.444933,0.955581,1.373105,1.674924,1.860948,1.949201,1.967517,1.945041,...,0.812599,1.149480,1.515640,1.854068,2.097038,2.176246,2.034426,1.636466,0.977889,0.089131
106,downstairs,0.108889,0.289262,0.426236,0.546627,0.677795,0.841012,1.046031,1.287271,1.542101,...,-1.836566,-0.318570,1.282153,2.840642,4.241250,5.386997,6.205115,6.649570,6.701688,6.369585


In [12]:
## ML portion starts here

X = training_data.drop(columns=['movement'])
y = training_data['movement']

X_train, X_valid, y_train, y_valid = train_test_split(X, y)

bayes_model = make_pipeline(
    GaussianNB()
)

knn_model = make_pipeline(
    StandardScaler(),
    KNeighborsClassifier(n_neighbors=5)
)

nn_model = make_pipeline(
    StandardScaler(),
    MLPClassifier(solver='lbfgs', hidden_layer_sizes=(
        16, 8, 4), activation='logistic', max_iter=100000)
)

dt_model = make_pipeline(
    StandardScaler(),
    DecisionTreeClassifier(max_depth=4)
)

rf_model = make_pipeline(
    RandomForestClassifier(n_estimators=1500, max_depth=3, min_samples_leaf=3)
)

en_model = make_pipeline(
    VotingClassifier([
        ('nb', GaussianNB()),
        ('knn', KNeighborsClassifier(5)),
        ('tree1', DecisionTreeClassifier(max_depth=4)),
        ('tree2', DecisionTreeClassifier(min_samples_leaf=3)),
        ('rf', RandomForestClassifier(n_estimators=250, max_depth=3, min_samples_leaf=2)),
        ('mlp', MLPClassifier(solver='lbfgs', hidden_layer_sizes=(16, 8, 6), activation='logistic', max_iter=100000))])

)

print('bayes')
bayes_model.fit(X_train, y_train)
print(bayes_model.score(X_train, y_train))
print(bayes_model.score(X_valid, y_valid))
y_predicted = bayes_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('knn')
knn_model.fit(X_train, y_train)
print(knn_model.score(X_train, y_train))
print(knn_model.score(X_valid, y_valid))
y_predicted = knn_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('neural net')
nn_model.fit(X_train, y_train)
print(nn_model.score(X_train, y_train))
print(nn_model.score(X_valid, y_valid))
y_predicted = nn_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('decision tree')
dt_model.fit(X_train, y_train)
print(dt_model.score(X_train, y_train))
print(dt_model.score(X_valid, y_valid))
y_predicted = dt_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('random forest')
rf_model.fit(X_train, y_train)
print(rf_model.score(X_train, y_train))
print(rf_model.score(X_valid, y_valid))
y_predicted = rf_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('ensemble')
en_model.fit(X_train, y_train)
print(en_model.score(X_train, y_train))
print(en_model.score(X_valid, y_valid))
y_predicted = en_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))


bayes
0.7037037037037037
0.6296296296296297
              precision    recall  f1-score   support

  downstairs       0.69      0.69      0.69        16
    upstairs       0.55      0.55      0.55        11

    accuracy                           0.63        27
   macro avg       0.62      0.62      0.62        27
weighted avg       0.63      0.63      0.63        27


knn
0.8024691358024691
0.7777777777777778
              precision    recall  f1-score   support

  downstairs       1.00      0.62      0.77        16
    upstairs       0.65      1.00      0.79        11

    accuracy                           0.78        27
   macro avg       0.82      0.81      0.78        27
weighted avg       0.86      0.78      0.78        27


neural net
0.9506172839506173
0.5555555555555556
              precision    recall  f1-score   support

  downstairs       0.64      0.56      0.60        16
    upstairs       0.46      0.55      0.50        11

    accuracy                           0.56  